# 1. Objective

Reproduce the R for marketing workshop from [here](https://bookdown.org/content/1340/data.html)

# 2. Imports

In [1]:
library(tidyverse)
library(ggpubr)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



# 3. Data

In [ ]:
download.file('https://s3.amazonaws.com/tomslee-airbnb-data-2/paris.zip',
              '~/ml-projects/regression-tutorial/paris-listings.zip')

## 3.1. Inspect one file

In [ ]:
paris_listings_df = read_csv('paris-listings/tomslee_airbnb_paris_1478_2017-07-25.csv')

In [ ]:
nrow(paris_listings_df)

In [ ]:
names(paris_listings_df)

In [ ]:
head(paris_listings_df)

In [ ]:
paris_listings_df %>% 
    group_by(neighborhood) %>% 
    summarize(mean_price = mean(price)) %>%
    arrange(desc(mean_price)) %>% 
    head(10) %>%
    pull(neighborhood) -> 
    most_expensive_neighborhoods

In [ ]:
ggplot(filter(paris_listings_df, neighborhood %in% most_expensive_neighborhoods), 
       aes(x = neighborhood, y = price)) +
    geom_jitter() +
    stat_summary(fun.y = median, colour = "tomato3", size = 4, geom = "point") +
    coord_flip() +
    theme_pubr()

## 3.2. Merge all the csv's in the directory

In [2]:
aggregated_paris_listings_df = data.frame()

for (csv_file in list.files('paris-listings')) {
    
    listing_df = read_csv(paste0('~/ml-projects/regression-tutorial/paris-listings/',csv_file))
    
    aggregated_paris_listings_df = bind_rows(aggregated_paris_listings_df, listing_df)

}

Parsed with column specification:
cols(
  room_id = col_double(),
  host_id = col_double(),
  room_type = col_character(),
  borough = col_logical(),
  neighborhood = col_character(),
  reviews = col_double(),
  overall_satisfaction = col_double(),
  accommodates = col_double(),
  bedrooms = col_double(),
  price = col_double(),
  minstay = col_double(),
  latitude = col_double(),
  longitude = col_double(),
  last_modified = col_datetime(format = "")
)
Parsed with column specification:
cols(
  room_id = col_double(),
  host_id = col_double(),
  room_type = col_character(),
  borough = col_logical(),
  neighborhood = col_character(),
  reviews = col_double(),
  overall_satisfaction = col_double(),
  accommodates = col_double(),
  bedrooms = col_double(),
  price = col_double(),
  minstay = col_double(),
  latitude = col_double(),
  longitude = col_double(),
  last_modified = col_datetime(format = "")
)
Parsed with column specification:
cols(
  room_id = col_double(),
  host_id = col_do

In [3]:
nrow(aggregated_paris_listings_df)

[1] 837830

# 4. Exploratory Analysis 

In [4]:
summary(aggregated_paris_listings_df)

    room_id            host_id           room_type         borough       
 Min.   :    2525   Min.   :     1415   Length:837830      Mode:logical  
 1st Qu.: 2642991   1st Qu.:  5519900   Class :character   NA's:837830   
 Median : 6633710   Median : 14118360   Mode  :character                 
 Mean   : 7483464   Mean   : 24040348                                    
 3rd Qu.:12297745   3rd Qu.: 33546017                                    
 Max.   :20144084   Max.   :143204619                                    
                    NA's   :289                                          
 neighborhood          reviews       overall_satisfaction  accommodates   
 Length:837830      Min.   :  0.00   Min.   :0.00         Min.   : 0.000  
 Class :character   1st Qu.:  0.00   1st Qu.:0.00         1st Qu.: 2.000  
 Mode  :character   Median :  3.00   Median :4.50         Median : 3.000  
                    Mean   : 12.64   Mean   :3.21         Mean   : 3.084  
                    3rd Qu.: 13.0

In [5]:
aggregated_paris_listings_df %>%
    mutate(room_id = factor(room_id),
           host_id = factor(host_id)) %>%
    select(-country, -survey_id, -bathrooms, -borough,
           -minstay, -location, -last_modified) ->
    aggregated_paris_listings_df

In [6]:
names(aggregated_paris_listings_df)

[1] "room_id"              "host_id"              "room_type"           
 [4] "neighborhood"         "reviews"              "overall_satisfaction"
 [7] "accommodates"         "bedrooms"             "price"               
[10] "latitude"             "longitude"            "city"                
[13] "name"                 "property_type"

In [7]:
aggregated_paris_listings_df %>%
    mutate(overall_satisfaction = replace(overall_satisfaction, overall_satisfaction == 0, NA)) ->
    aggregated_paris_listings_df

# 5. Hypothesis testing

### 5.1. Mean of price is different beween smallest and largest neighborhoods, i.e., larger the neighborhood, higher the price

In [ ]:
print(unique(aggregated_paris_listings_df$neighborhood))

In [ ]:
aggregated_paris_listings_df %>%
    filter(neighborhood == "Champs-Elysées" | neighborhood == "Sorbonne") %>%
    group_by(neighborhood) %>% 
    summarize(n_obs = n(),
              mean_price = mean(price),
              sd_price = sd(price),
              median_price = median(price))

In [ ]:
paris_subset_df = filter(aggregated_paris_listings_df, neighborhood == "Champs-Elysées" | neighborhood == "Sorbonne")

In [ ]:
nrow(paris_subset_df)

In [ ]:
t.test(price ~ neighborhood, 
       data =  paris_subset_df, 
       var.equal = FALSE)

The null hypothesis was that the mean price of Champs Elysees is the same as the mean price of Sorbonne, and this was rejected

### 5.2. Mean prices differ by apartments of different room types

In [ ]:
unique(aggregated_paris_listings_df$room_type)

In [ ]:
aggregated_paris_listings_df %>%
    group_by(room_type) %>%
    summarize(n_samples = n(),
              mean_price = mean(price),
              sd_price = sd(price),
              median_price = median(price))

In [ ]:
aggregated_paris_listings_df %>%
    filter(!is.na(room_type)) %>% 
    group_by(room_type) %>%
    summarize(n_samples = n(),
              mean_price = mean(price),
              sd_price = sd(price),
              median_price = median(price))

It seems that shared rooms are priced lower than private rooms which are priced lower than entire homes. However, many more entire homes are available compared to private and shared rooms. 

If price is the dependent variable, a cursory glance at whether it resembles a normal distribution is required.

In [ ]:
ggplot(filter(aggregated_paris_listings_df, !is.na(room_type))) +
    facet_wrap(~ room_type, ncol = 1) + 
    geom_histogram(aes(x = price))

There is an evident right skew on price. This normality can be formally tested.

In [ ]:
for (room_type_str in unique(filter(aggregated_paris_listings_df, !is.na(room_type))$room_type)) {
    
    cat("Testing normality of price for", room_type_str)
    
    aggregated_paris_listings_df %>%
    filter(room_type == room_type_str) %>%
    pull(price) %>%
    nortest::ad.test() %>%
    print()

}

Evidently, the normlity assumption is invalid

In [ ]:
ggplot(filter(aggregated_paris_listings_df, !is.na(room_type))) +
    facet_wrap(~ room_type, ncol = 1) + 
    geom_histogram(aes(x = log(price)))

Log transform improves the situation. Next we wish to see if the groups have equal variance

In [ ]:
model_price_room_type = lm(price ~ room_type, 
                           data = filter(aggregated_paris_listings_df, !is.na(room_type)))

In [ ]:
type3anova::type3anova(model_price_room_type)

This ANOVA indicates that price of rooms is different across room types

In [ ]:
TukeyHSD(aov(price ~ room_type, 
             data = filter(aggregated_paris_listings_df, !is.na(room_type))))

This indicates that prices across all the three room types is different

### 5.3. Price is driven by the overall satisfaction derived

In [ ]:
ggplot(filter(aggregated_paris_listings_df, !is.na(room_type)),
       aes(x = overall_satisfaction, y = log(price))) +
    geom_jitter(alpha = 1/10) +
    stat_summary(fun.y = mean,
                 color = 'steelblue',
                 geom = 'point',
                 size = 4) +
    stat_smooth(method = 'lm', se = FALSE) # add in the regression line

In [ ]:
model_price_satisfaction = lm(price ~ overall_satisfaction,
                              data = filter(aggregated_paris_listings_df, !is.na(room_type)))

In [ ]:
summary(model_price_satisfaction)

Very low R squared indicates that overall satisfaction does not explain the variation in price. We will need to look for other predictors

### 5.4. What drives prices of properties?

In [ ]:
aggregated_paris_listings_df = filter(aggregated_paris_listings_df, !is.na(room_type))

In [ ]:
aggregated_paris_listings_df %>%
    select(price, overall_satisfaction, reviews, accommodates) %>%
    drop_na() %>%
    summary()

In [ ]:
aggregated_paris_listings_df %>%
    select(price, overall_satisfaction, reviews, accommodates) %>%
    drop_na() %>%
    cor()

These seem to be nice features

In [ ]:
lm(price ~ overall_satisfaction + reviews + accommodates,
   data = aggregated_paris_listings_df) %>%

    summary()

In [ ]:
lm(log(price) ~ overall_satisfaction + reviews + accommodates,
   data = aggregated_paris_listings_df) %>%
    
    summary()

In [ ]:
lm(log(price) ~ overall_satisfaction * reviews,
   data = aggregated_paris_listings_df) %>% 
    
    summary()

In [ ]:
lm(log(price) ~ overall_satisfaction * reviews + accommodates,
   data = aggregated_paris_listings_df) %>% 
    
    summary()

In [ ]:
lm(log(price) ~ overall_satisfaction * reviews * accommodates,
   data = aggregated_paris_listings_df) %>% 
    
    summary()

In [ ]:
aggregated_paris_listings_df %>%
    filter(!is.na(overall_satisfaction)) %>%
    summarize(q1 = quantile(reviews, 0.33),
              q2 = quantile(reviews, 0.66),
              max = max(reviews))

In [ ]:
aggregated_paris_listings_df %>%
    filter(!is.na(overall_satisfaction)) %>%
    mutate(review_group = case_when(reviews <= quantile(reviews, 0.33) ~ 'low',
                                    reviews <= quantile(reviews, 0.66) ~ 'medium',
                                    TRUE ~ 'high'),
          review_group = factor(review_group, levels = c("low","medium","high"))) ->
    aggregated_paris_reviews_df

In [ ]:
aggregated_paris_reviews_df %>%
    group_by(review_group) %>%
    summarize(min = min(reviews),
              max = max(reviews))

In [ ]:
ggplot(aggregated_paris_reviews_df,
       aes(x = overall_satisfaction, y = log(price))) + 
    facet_wrap(~ review_group) +
    geom_jitter(color = 'lightgrey', alpha = 1/10) +
    stat_smooth(method = "lm", se= FALSE)

### 5.4. Number of best places to stay varies by neighborhood

In [15]:
aggregated_paris_listings_df %>%
    mutate(gem = (overall_satisfaction == 5) & (reviews >=30),
           gem = factor(gem, labels = c('no gem', 'gem'))) %>%
    filter(!is.na(gem)) ->
aggregated_paris_listings_df

In [16]:
aggregated_paris_listings_df %>%
    filter(neighborhood == "Champs-Elysées" | neighborhood == "Sorbonne") %>%
    group_by(neighborhood, gem) %>%
    summarize(n_samples = n())

neighborhood,gem,n_samples
<chr>,<fct>,<int>
Champs-Elysées,no gem,3516
Champs-Elysées,gem,227
Sorbonne,no gem,7127
Sorbonne,gem,449


We need to build a cross-tab for these two cities to test if gems are really more in Sorbone

In [18]:
aggregated_paris_listings_df %>%
    filter(neighborhood == "Champs-Elysées" | neighborhood == "Sorbonne", !is.na(gem)) ->
    subset_paris_listings_df

cross_tab_cities = table(subset_paris_listings_df$neighborhood,
                         subset_paris_listings_df$gem)

In [19]:
prop.table(cross_tab_cities)

                
                     no gem        gem
  Champs-Elysées 0.31062815 0.02005478
  Sorbonne       0.62964926 0.03966782

In [20]:
chisq.test(cross_tab_cities)


	Pearson's Chi-squared test with Yates' continuity correction

data:  cross_tab_cities
X-squared = 0.062209, df = 1, p-value = 0.803


We cannot reject the null that the number of gems is no different between Sorbonne and Champs-Elysees!

### 5.5. Predicting number of best places

In [23]:
model_logit = glm(gem ~ price * room_type,
                  data = aggregated_paris_listings_df,
                  family = "binomial")

In [24]:
summary(model_logit)


Call:
glm(formula = gem ~ price * room_type, family = "binomial", data = aggregated_paris_listings_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3107  -0.2702  -0.2652  -0.2630   2.7850  

Coefficients:
                              Estimate Std. Error  z value Pr(>|z|)    
(Intercept)                 -3.383e+00  7.865e-03 -430.174   <2e-16 ***
price                        5.282e-04  3.438e-05   15.364   <2e-16 ***
room_typePrivate room        2.689e-01  2.337e-02   11.505   <2e-16 ***
room_typeShared room        -2.240e-02  1.544e-01   -0.145    0.885    
price:room_typePrivate room -3.606e-04  2.267e-04   -1.591    0.112    
price:room_typeShared room  -4.749e-03  3.116e-03   -1.524    0.128    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 256701  on 829168  degrees of freedom
Residual deviance: 256285  on 829163  degrees of freedom
  (301 observations del

In [29]:
aggregated_paris_listings_df %>%
    mutate(gem_pred_logit = predict(model_logit, aggregated_paris_listings_df),
           gem_odds = exp(gem_pred_logit),
           gem_prob_pred = gem_odds/(1+gem_odds),
           gem_pred = case_when(gem_prob_pred <= 0.5 ~ 'no gem',
                                gem_prob_pred > 0.5 ~ 'gem')) ->
    aggregated_paris_listings_df

In [31]:
table(aggregated_paris_listings_df$gem, aggregated_paris_listings_df$gem_pred)

        
            gem no gem
  no gem     25 799350
  gem         0  29794

In [32]:
cat("Model Accuracy: ", (799350)/(799350+25+29794))

Model Accuracy:  0.9640375